In [1]:
# Import Libraries
import numpy as np
import pandas as pd 
import yfinance as yf

Recap : The Taylor rule states that the central bank of each country decides its nominal interest rate as a linear function of its own inflation rate and GDP gap, i.e. $i^{us}_t = \alpha^{us} + \beta^{us}_1\pi^{us}_t + \beta^{us}_2 (y^{us}_t - \bar{y}^{us})$

Also, theoretically, we know that the change in the interest rate differential affect exchange rates. Thus, assuming the Taylor rule is true, the MP (2009) paper builds a forecasting model for log nominal exchange rate using the Taylor rule fundamental varables in two countries, which are inflation rates and GDP gaps. 

The simplest specification of the exchange rate forecasting equation used in MP is as follows:

$$s_{t+1} - s_t = \Delta s_{t+1} = \beta_1 + \beta_2 (\pi^{us}_t - \pi^{ja}_t) + \beta_3 (\text{gap}^{us}_t - \text{gap}^{ja}_t) + \epsilon_t $$

This specification is called a "homogenous, symmetric Taylor rule model without smoothing" in the paper.

### 1. Industrial Production data

In [2]:
df_original = pd.read_csv('final_project_data.csv', parse_dates= True, index_col=0)
df_original

,USA2YR,GTGBP2YR,UKRPCJYR Index,CPI YOY Index,IP Index,UKIPI Index
Date,,,,,,
1989-03-31,9.668,NaN,NaN,NaN,NaN,NaN
1989-04-03,9.528,NaN,NaN,NaN,NaN,NaN
1989-04-04,9.405,NaN,NaN,NaN,NaN,NaN
1989-04-05,9.491,NaN,NaN,NaN,NaN,NaN
1989-04-06,9.526,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-12-27,4.244,3.955,NaN,NaN,NaN,NaN
2023-12-28,4.278,3.979,NaN,NaN,NaN,NaN
2023-12-29,4.251,3.955,NaN,NaN,NaN,NaN


In [3]:
# Resample Monthly
df_original_monthly = df_original.resample('M').last()

In [4]:
# Limit to Industrial production Data
USA_Ind_Prod = df_original_monthly['IP Index']
UK_Ind_Prod = df_original_monthly['UKIPI Index']

Ind_Prod_data = pd.merge(USA_Ind_Prod, UK_Ind_Prod, left_index=True, right_index=True)
Ind_Prod_data

,IP Index,UKIPI Index
Date,,
1989-03-31,NaN,NaN
1989-04-30,62.0592,74.3
1989-05-31,62.0424,76.2
1989-06-30,61.6948,74.2
1989-07-31,61.6915,74.2
...,...,...
2023-09-30,103.2411,95.0
2023-10-31,103.3346,94.9
2023-11-30,102.4592,93.7


### 2. Inflation data

In [5]:
df_USA_PCE = pd.read_excel("PCEPILFE.xls", parse_dates= True, index_col=0)
df_GB_PCE = pd.read_excel("GBRCPICORMINMEI.xls", parse_dates= True, index_col=0)

In [6]:
# Merge df_exch_rates and df_USA_PCE on 'Date' (drops Dec 2023 bc not in UK data)
PCE_data = pd.merge(df_USA_PCE, df_GB_PCE, left_index=True, right_index=True, how='inner')

# Resample to convert to end of month
PCE_data = PCE_data.resample('M').last()
PCE_data

,USA_PCE,UK_PCE
Date,,
1989-01-31,8.40153,8.378510
1989-02-28,8.27590,8.759021
1989-03-31,7.30869,8.729165
1989-04-30,8.58106,8.622486
1989-05-31,7.83945,8.797702
...,...,...
2023-07-31,5.91807,6.400000
2023-08-31,5.42273,5.900000
2023-09-30,5.54651,5.900000


### 3. Exchange Rate data

In [7]:
# Daily exchange rate data
exchange_rate_usd_gbp = pd.read_excel('DEXUSUK.xls', parse_dates=['Date'], dtype={'DEXUSUK': float})

# Set the 'DATE' column as the index
exchange_rate_usd_gbp.set_index('Date', inplace=True)

# Get the closing values of the last day of each month
exrates_closing_values = exchange_rate_usd_gbp.resample('M').last()

exrates_closing_values

,DEXUSUK
Date,
1989-02-28,1.7440
1989-03-31,1.6852
1989-04-30,1.6890
1989-05-31,1.5715
1989-06-30,1.5490
...,...
2023-10-31,1.2135
2023-11-30,1.2640
2023-12-31,1.2743


In [8]:
# Calculate Difference in Exchange Rates and Shift
exrates_closing_values['s_diff_current'] = exrates_closing_values['DEXUSUK'].diff()
exrates_closing_values['s_diff_future'] = exrates_closing_values['s_diff_current'].shift(-1)
exrates_closing_values

,DEXUSUK,s_diff_current,s_diff_future
Date,,,
1989-02-28,1.7440,NaN,-0.0588
1989-03-31,1.6852,-0.0588,0.0038
1989-04-30,1.6890,0.0038,-0.1175
1989-05-31,1.5715,-0.1175,-0.0225
1989-06-30,1.5490,-0.0225,0.1170
...,...,...,...
2023-10-31,1.2135,-0.0079,0.0505
2023-11-30,1.2640,0.0505,0.0103
2023-12-31,1.2743,0.0103,-0.0010


In [10]:
# Join Datasets
final_df = pd.merge(exrates_closing_values, PCE_data, left_index=True, right_index=True, how='inner')
final_df = pd.merge(final_df, Ind_Prod_data, left_index=True, right_index=True, how='inner')
final_df

,DEXUSUK,s_diff_current,s_diff_future,USA_PCE,UK_PCE,IP Index,UKIPI Index
Date,,,,,,,
1989-03-31,1.6852,-0.0588,0.0038,7.30869,8.729165,NaN,NaN
1989-04-30,1.6890,0.0038,-0.1175,8.58106,8.622486,62.0592,74.3
1989-05-31,1.5715,-0.1175,-0.0225,7.83945,8.797702,62.0424,76.2
1989-06-30,1.5490,-0.0225,0.1170,7.47627,8.777054,61.6948,74.2
1989-07-31,1.6660,0.1170,-0.0930,7.26957,8.824701,61.6915,74.2
...,...,...,...,...,...,...,...
2023-07-31,1.2857,0.0148,-0.0190,5.91807,6.400000,102.3002,96.4
2023-08-31,1.2667,-0.0190,-0.0453,5.42273,5.900000,103.1900,95.4
2023-09-30,1.2214,-0.0453,-0.0079,5.54651,5.900000,103.2411,95.0


In [12]:
final_df.to_csv('MP_Strategy_Data.csv')